In [47]:
from itertools import count, cycle, islice
from collections import namedtuple, defaultdict, deque, Counter, defaultdict

%load_ext line_profiler
# usage: %lprun -f func main

def Input(day):
    "Open this day's input file."
    filename = 'input{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        print("No input file found for day", day)
        
def consume(iterator, n):
    "Advance the iterator n-steps ahead. If n is none, consume entirely."
    # Use functions that consume iterators at C speed.
    if n is None:
        # feed the entire iterator into a zero-length deque
        collections.deque(iterator, maxlen=0)
    else:
        # advance to the empty slice starting at position n
        next(islice(iterator, n, n), None)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha
### Part one
Find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

In [2]:
def parse(text):
    "Parse a string of numbers into a list of digits"
    return [int(n) for n in list(text.strip())]

def captcha_n(digits, k):
    sum = 0
    
    for i, n in enumerate(digits):
        if n == digits[(i + k) % len(digits)]:
            sum += n
    
    return sum

def captcha(digits):
    return captcha_n(digits, 1)

assert parse("1234") == [1, 2, 3, 4]
assert captcha([1, 1, 2, 2]) == 3
assert captcha([1, 1, 1, 1]) == 4
assert captcha([1, 2, 3, 4]) == 0
assert captcha([9, 1, 2, 1, 2, 1, 2, 9]) == 9

captcha(parse(Input(1).read()))

1034

#### Part two
Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list.

In [3]:
def captcha_halfway(digits):
    assert len(digits) % 2 == 0
    return captcha_n(digits, len(digits) // 2)

assert captcha_halfway([1, 2, 1, 2]) == 6
assert captcha_halfway([1, 2, 3, 4, 2, 5]) == 4
assert captcha_halfway([1, 2, 3, 1, 2, 3]) == 12
assert captcha_halfway([1, 2, 1, 3, 1, 4, 1, 5]) == 4

captcha_halfway(parse(Input(1).read()))

1356

# [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum
### Part one
For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

In [4]:
def max_range(row):
    low = float("inf")
    high = 0
    
    for i in row:
        if i < low:
            low = i
        if i > high:
            high = i
    
    return high - low

def checksum(file, row_func):
    sum = 0
    for row in file:
        sum += row_func(row)
        
    return sum

def parse(text):
    "Parse lines of tab separated numbers into lists of integers"
    res = []
    for line in text.strip().split("\n"):
        res.append([int(n) for n in line.split("\t")])
    return res

assert parse("1236\t741\t557\t1029\t144") == [[1236, 741, 557, 1029, 144]]
test_input = [[5, 1, 9, 5], [7, 5, 3], [2, 4, 6, 8]]
assert checksum(test_input, max_range) == 18

checksum(parse(Input(2).read()), max_range)

42299

### Part two
It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

In [5]:
def quotient(row):
    row.sort()
    for i, n in enumerate(row):
        for m in row[:i:-1]:
            if m % n == 0:
                return m // n

assert checksum([[5, 9, 2, 8]], quotient) == 4

checksum(parse(Input(2).read()), quotient)

277

# [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory
### Part one

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```
While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

Data from square 1 is carried 0 steps, since it's at the access port.
Data from square 12 is carried 3 steps, such as: down, left, left.
Data from square 23 is carried only 2 steps: up twice.
Data from square 1024 must be carried 31 steps.
How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

---

So, I could either build out the entire spiral and walk it to find the result, _or_ I could just do some math. We could find the corner nearest to our destination, and easily compute how many steps away that is from the center. It's then easy to see how many steps the corner is from our destination. In other words, 
``` 
distance(center, destination) == distance(center, nearest_corner) - distance(nearest_corner, destination)
```
Math sounds faster. 

In [6]:
def center_distance(n):
    "Given a number, return its distance from the center of the spiral map"
    box_height = 1
    x = 0
    
    # Find box height
    while x < n:
        box_height += 2
        x = box_height * box_height
    
    # Find nearest corner
    corner = x
    while corner > n:
        corner -= box_height - 1
    
    if n - corner > corner + (box_height - 1) - n:
        corner = corner + (box_height - 1)
    
    # Distance from center to nearest corner - distance from nearest corner to point
    diag = (box_height // 2) * 2
    print(diag - abs(corner - n))

center_distance(265149)


438


### Part 2
As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

Square 1 starts with the value 1.
Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.
Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```
What is the first value written that is larger than your puzzle input?

---

Whoops, should have built the spiral.

In [7]:
class SpiralLocation:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.value = 1

def get_neighbors(x, y):
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))
        
def spiral(end, cell_func):
    "Build a spiraled map from 1..end, setting the value at each cell with cell_func"
    loc = SpiralLocation()
    width = 1   
    spiral = {(loc.x, loc.y): loc.value}

    down = lambda x: x - 1
    up = lambda x: x + 1
    
    def spiral_side(axis, direction):
        corner = max_coord if direction == up else -max_coord
        current = getattr(loc, axis)

        while current != corner:
            setattr(loc, axis, direction(current))
            loc.value = cell_func(loc, spiral)
            spiral[(loc.x, loc.y)] = loc.value
            if loc.value >= end:
                return True
            current = getattr(loc, axis)
        return False
        
    while loc.value < end:
        width += 2
        max_coord = width // 2
        
        if spiral_side("x", up):
            return spiral
        if spiral_side("y", up):
            return spiral
        if spiral_side("x", down):
            return spiral
        if spiral_side("y", down):
            return spiral
        if spiral_side("x", up):
            return spiral
            
    return spiral

def neighbor_sum(loc, spiral):
    res = 0
    neighbor_coords = get_neighbors(loc.x, loc.y)
    for n in neighbor_coords:
        value = spiral.get(n)
        if value:
            res += value

    return res
 
res = spiral(25, lambda loc, spiral: loc.value + 1)
assert res[(0, 2)] == 15
assert res[(0, 0)] == 1
assert res[(2, -2)] == 25
assert res[(-1, 0)] == 6

test_spiral = {(0, 1): 4, (-1, 1): 5, (0, 0): 1, (-1, 0): 6, (-1, -1): 7, (0, -1): 8, (1, 0): 2, (1, -1): 9, (1, 1): 3}
test_loc = SpiralLocation()
test_loc.x = -1
test_loc.y = 0
assert neighbor_sum(test_loc, test_spiral) == 25

neighbor_spiral = spiral(750, neighbor_sum)
assert sorted(list(neighbor_spiral.values()))[-1] == 806

neighbor_spiral = spiral(265149, neighbor_sum)
sorted(list(neighbor_spiral.values()))[-1]

266330

# [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases

### Part one
A new system policy has been put in place that requires all accounts to use a passphrase instead of simply a password. A passphrase consists of a series of words (lowercase letters) separated by spaces.

To ensure security, a valid passphrase must contain no duplicate words.

For example:
```
aa bb cc dd ee is valid.
aa bb cc dd aa is not valid - the word aa appears more than once.
aa bb cc dd aaa is valid - aa and aaa count as different words.
```
The system's full passphrase list is available as your puzzle input. How many passphrases are valid?

In [8]:
def no_dupes(words):
    return len(words) - len(set(words)) == 0

def parse_words(text):
    return text.split(" ")

def valid_passphrase_count(lines, filter_func):
    parsed_lines = map(parse_words, lines)
    return len(list(filter(filter_func, parsed_lines)))


assert no_dupes(["aa", "bb", "cc", "dd", "ee"]) == True
assert no_dupes(["aa", "bb", "cc", "dd", "aa"]) == False
assert no_dupes(["aa", "bb", "cc", "dd", "aaa"]) == True
assert parse_words("aa bb cc") == ["aa", "bb", "cc"]

valid_passphrase_count(Input(4).read().splitlines(), no_dupes)

383

### Part two
For added security, yet another system policy has been put in place. Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

For example:
```
abcde fghij is a valid passphrase.
abcde xyz ecdab is not valid - the letters from the third word can be rearranged to form the first word.
a ab abc abd abf abj is a valid passphrase, because all letters need to be used when forming another word.
iiii oiii ooii oooi oooo is valid.
oiii ioii iioi iiio is not valid - any of these words can be rearranged to form any other word.
```
Under this new system policy, how many passphrases are valid?

In [9]:
def no_anagrams(words):
    sorted_words = [''.join(s) for s in list(map(sorted, words))]
    return no_dupes(sorted_words)

assert no_anagrams(["cat", "act"]) == False
valid_passphrase_count(Input(4).read().splitlines(), no_anagrams)

265

# [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike

### Part one
An urgent interrupt arrives from the CPU: it's trapped in a maze of jump instructions, and it would like assistance from any programs with spare cycles to help find the exit.

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

For example, consider the following list of jump offsets:

```
0
3
0
1
-3
```
Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:

```
(0) 3  0  1  -3  - before we have taken any steps.
(1) 3  0  1  -3  - jump with offset 0 (that is, don't jump at all). Fortunately, the instruction is then incremented to 1.
 2 (3) 0  1  -3  - step forward because of the instruction we just modified. The first instruction is incremented again, now to 2.
 2  4  0  1 (-3) - jump all the way to the end; leave a 4 behind.
 2 (4) 0  1  -2  - go back to where we just were; increment -3 to -2.
 2  5  0  1  -2  - jump 4 steps forward, escaping the maze.
 ```
In this example, the exit is reached in 5 steps.

How many steps does it take to reach the exit?

In [10]:
part_one = lambda x: x + 1

def trampoline(jumps, rule):
    loc = 0
    jump_count = 0
    while True:
        # index of next hop
        move = loc + jumps[loc]
        if move < 0 or move >= len(jumps):
            return jump_count + 1
        jumps[loc] = rule(jumps[loc])
        jump_count += 1
        loc = move

assert trampoline([0, 3, 0, 1, -3], part_one) == 5
trampoline([int(n) for n in Input(5).readlines()], part_one)

351282

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

Using this rule with the above example, the process now takes 10 steps, and the offset values after finding the exit are left as 2 3 2 3 -1.

How many steps does it now take to reach the exit?

In [11]:
part_two = lambda x: x - 1 if x > 2 else x + 1
assert trampoline([0, 3, 0, 1, -3], part_two) == 10
trampoline([int(n) for n in Input(5).readlines()], part_two)

24568703

# [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation

### Part one

A debugger program here is having an issue: it is trying to repair a memory reallocation routine, but it keeps getting stuck in an infinite loop.

In this area, there are sixteen memory banks; each memory bank can hold any number of blocks. The goal of the reallocation routine is to balance the blocks between the memory banks.

The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

For example, imagine a scenario with only four memory banks:

- The banks start with 0, 2, 7, and 0 blocks. 
- The third bank has the most blocks, so it is chosen for redistribution.
- Starting with the next bank (the fourth bank) and then continuing to the first bank, the second bank, and so on, the 7 blocks are spread out over the memory banks. The fourth, first, and second banks get two blocks each, and the third bank gets one back. The final result looks like this: 2 4 1 2.
- Next, the second bank is chosen because it contains the most blocks (four). Because there are four memory banks, each gets one block. The result is: 3 1 2 3.
- Now, there is a tie between the first and fourth memory banks, both of which have three blocks. The first bank wins the tie, and its three blocks are distributed evenly over the other three banks, leaving it with none: 0 2 3 4.
- The fourth bank is chosen, and its four blocks are distributed such that each of the four banks receives one: 1 3 4 1.
 -The third bank is chosen, and the same thing happens: 2 4 1 2.
 -At this point, we've reached a state we've seen before: 2 4 1 2 was already seen. The infinite loop is detected after the fifth block redistribution cycle, and so the answer in this example is 5.

Given the initial block counts in your puzzle input, how many redistribution cycles must be completed before a configuration is produced that has been seen before?

In [12]:
INPUT = "0    5    10    0    11    14    13    4    11    8    8    7    1    4    12    11"

def redistribute(banks):
    block_count = max(banks)
    loc = banks.index(block_count)
    banks[loc] = 0
    
    while block_count > 0:
        loc = (loc + 1) % len(banks)
        banks[loc] += 1
        block_count -= 1

b = [0, 2, 7, 0]
redistribute(b)
assert b == [2, 4, 1, 2]

def find_loop(banks):
    seen_configs = set()
    cycles = 0
    
    while True:
        config = ".".join([str(n) for n in banks])
        if config in seen_configs:
            return cycles
        seen_configs.add(config)
        redistribute(banks)
        cycles += 1
    
assert find_loop([0, 2, 7, 0]) == 5

find_loop([int(n) for n in INPUT.split("    ")])

7864

### Part two

Out of curiosity, the debugger would also like to know the size of the loop: starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

In the example above, 2 4 1 2 is seen again after four cycles, and so the answer in that example would be 4.

How many cycles are in the infinite loop that arises from the configuration in your puzzle input?

In [13]:
def find_loop_size(banks):
    seen_configs = {}
    cycles = 0
    
    while True:
        config = ".".join([str(n) for n in banks])
        if config in seen_configs:
            return cycles - seen_configs[config]
        seen_configs[config] = cycles
        redistribute(banks)
        cycles += 1

assert find_loop_size([0, 2, 7, 0]) == 4
find_loop_size([int(n) for n in INPUT.split("    ")])

1695

# [Day 7](http://adventofcode.com/2017/day/7): Recursive Circus

### Part one

In [14]:
test_input = """pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)"""

class Disc:
    def __init__(self, name, weight, children):
        self.name = name
        self.weight = weight
        self.children = children
        self.original_weight = weight

def parse_disc(text):
    parts = text.strip().split(" ")
    name, weight, *_ = parts
    weight = int(weight[1:-1])
    children = [child.strip(',') for child in parts[3:]]
    return Disc(name, weight, children)

test_disc = parse_disc(test_input.split("\n")[7])
assert test_disc.name == 'padx'
assert test_disc.weight == 45
assert test_disc.children == ['pbga', 'havc', 'qoyq']

def find_root(discs):
    disc_names = {disc.name for disc in discs}
    children = {child for disc in discs for child in disc.children}
    return disc_names.difference(children).pop()
    
test_discs = [parse_disc(t) for t in test_input.split("\n")]
assert find_root(test_discs) == 'tknk'

discs = [parse_disc(t) for t in Input(7).readlines()]
find_root(discs)

'vmpywg'

### Part two

In [15]:
def build_tree(root, discs):
    root.children = [discs[name] for name in root.children]
    for child in root.children:
        build_tree(child, discs)

def parse_discs(text):
    discs = [parse_disc(t) for t in text.split("\n")]
    root = find_root(discs)
    disc_map = {disc.name: disc for disc in discs}
    build_tree(disc_map[root], disc_map)
    return disc_map[root]

def build_weights(root, path=[]):
    path.append(root)
    for child in root.children:
        for node in path:
            node.weight += child.weight
        build_weights(child)
    path.pop()
        
root = parse_discs(test_input)
build_weights(root)
assert root.weight == 778
assert root.children[0].weight == 251
assert root.children[1].weight == 243
assert root.children[0].children[0].weight == 61    
    
def correct_wrong(weighted_root):
    weights = {}
    for child in weighted_root.children:
        if weights.get(child.weight):
            weights[child.weight].append(child)
        else:
            weights[child.weight] = [child]

    outlier = None
    for groups in weights.values():
        if len(groups) == 1:
            outlier = groups[0]
        else:
            matching = groups[0]
            
    if len(set(c.weight for c in outlier.children)) == 1:
        return outlier.original_weight - (outlier.weight - matching.weight)
    else:
        return correct_wrong(outlier)

assert correct_wrong(root) == 60

root = parse_discs(Input(7).read().strip())
build_weights(root)
print(correct_wrong(root))

1674


# [Day 8](http://adventofcode.com/2017/day/8): I Heard You Like Registers

Using Python for this feels like cheating...

In [16]:
test_input = """b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10"""

operation = {
    "inc": lambda x, y: x + y,
    "dec": lambda x, y: x - y
}

condition = {
    ">": lambda x, y: x > y,
    "<": lambda x, y: x < y,
    ">=": lambda x, y: x >= y,
    "<=": lambda x, y: x <= y,
    "==": lambda x, y: x == y,
    "!=": lambda x, y: x != y
}

def parse_reg(line, regs):
    target, symbol, amount, _, cond_target, cond_symbol, cond_amount = line.split()
    ret = 0
    if condition[cond_symbol](regs[cond_target], int(cond_amount)):
        regs[target] = operation[symbol](regs[target], int(amount))
        ret = regs[target]
    return ret

def max_reg(text):
    historical_max = 0
    regs = defaultdict(int)
    for line in text.splitlines():
        historical_max = max(historical_max, parse_reg(line, regs))
    return max(regs.values()), historical_max

assert max_reg(test_input) == (1, 10)

max_reg(Input(8).read())

(4647, 5590)

# [Day 9](http://adventofcode.com/2017/day/9): Stream Processing

Today was my first time being bitten by mutable default args 😱

I thought building a tree of the input might be useful for whatever Part 2 would entail, but that would up being straightforward.

In [17]:
class Node:
    def __init__(self, depth):
        self.depth = depth
        self.children = []
        
def parse_garbage(it):
    count = 0
    for c in it:
        if c == '>':
            return count
        if c == '!':
            next(it)
            continue
        count += 1

def parse_stream(it, root=None):
    if root == None:
        root = Node(0)
    for c in it:
        if c == '<':
            parse_garbage(it)
        elif c == '{':
            root.children.append(parse_stream(it, Node(root.depth + 1)))
        elif c == '}':
            return root
            
    return root
        
def walk_tree(root):
    score = 0
    nodes = deque([root])
    while nodes:
        node = nodes.popleft()
        score += node.depth
        for child in node.children:
            nodes.append(child)
    return score

def find_score(text):
    return walk_tree(parse_stream(iter(text)))

assert find_score("{}") == 1
assert find_score("{{{}}}") == 6
assert find_score("{{},{}}") == 5
assert find_score("{{{},{},{{}}}}") == 16.
assert find_score("{<a>,<a>,<a>,<a>}") == 1
assert find_score("{{<ab>},{<ab>},{<ab>},{<ab>}}") == 9
assert find_score("{{<!!>},{<!!>},{<!!>},{<!!>}}") == 9
assert find_score("{{<a!>},{<a!>},{<a!>},{<ab>}}") == 3

find_score(Input(9).read())

12803

In [18]:
def count_garbage(it):
    count = 0
    for c in it:
        if c == '<':
            count += parse_garbage(it)
    return count

assert count_garbage(iter('<{o"i!a,<{i<a>')) == 10

count_garbage(iter(Input(9).read()))

6425

# [Day 10](http://adventofcode.com/2017/day/10): Knot hash

In hindsight, itertools.cycle would have been real useful here.

In [19]:
INPUT = [120,93,0,90,5,80,129,74,1,165,204,255,254,2,50,113]

def ring_get(ring, n, i):
    "Get n elements from the ring starting at index i"
    to_end = ring[i:min(n + i, len(ring))]
    wrap = ring[:n - len(to_end)]
    return to_end + wrap
    
def ring_put(ring, elements, i):
    "Insert elements in ring, starting at index i"
    for j, e in enumerate(elements):
        idx = (i + j) % len(ring)
        ring[idx] = e
    return ring
    
def list_gen(n):
    "Generate a list of numbers from 0..n"
    return [i for i in range(n + 1)]

def twist(ring, length, i):
    to_reverse = ring_get(ring, length, i)
    to_reverse.reverse()
    ring_put(ring, to_reverse, i)
    return ring
    
def do_twists(ring, lengths, pos=0, skip_size=0):
    for l in lengths:
        twist(ring, l, pos)
        pos = (pos + l + skip_size) % len(ring)
        skip_size += 1
    return (ring, pos, skip_size)
    
l = [1, 2, 3, 4]
assert ring_get(l, 4, 2) == [3, 4, 1, 2]
assert ring_put(l, [6, 7, 8], 2) == [8, 2, 6, 7]
assert list_gen(4) == [0, 1, 2, 3, 4]

test_input = [3, 4, 1, 5]
test_list = list_gen(4)
assert twist(test_list, 3, 0) == [2, 1, 0, 3, 4]

assert do_twists(list_gen(4), test_input)[0] == [3, 4, 2, 1, 0]

first, second, *_ = do_twists(list_gen(255), INPUT)[0]
first * second

826

In [20]:
SUFFIX = [17, 31, 73, 47, 23]
ASCII_INPUT = "120,93,0,90,5,80,129,74,1,165,204,255,254,2,50,113"
RING_SIZE = 255
ITERATIONS = 64
BLOCK = 16

def hex_format(c):
    res = format(c, 'x')
    if len(res) == 1:
        return "0" + res
    return res

def knot_hash(s):
    lengths = [ord(c) for c in s] + SUFFIX
    pos = 0
    skip_size = 0
    sparse_hash = list_gen(RING_SIZE)
    for i in range(0, ITERATIONS):
        sparse_hash, pos, skip_size = do_twists(sparse_hash, lengths, pos, skip_size)
    
    dense_hash = []
    chunks = [sparse_hash[i:i + BLOCK] for i in range(0, len(sparse_hash), BLOCK)]
    for chunk in chunks:
        res = 0
        for el in chunk:
            res = res ^ el
        dense_hash.append(hex_format(res))
    
    return "".join(dense_hash)

assert knot_hash("") == "a2582a3a0e66e6e86e3812dcb672a272"
assert knot_hash("AoC 2017") == "33efeb34ea91902bb2f59c9920caa6cd"

knot_hash(ASCII_INPUT)

'd067d3f14d07e09c2e7308c3926605c4'

# [Day 11](http://adventofcode.com/2017/day/11): Hex Ed

Approaching part 1 by enumerating all direction combinations that cancel out or reduce to a simpler direction.

In [21]:
def shortest_path(hex_path):    
    def equivalent(d1, d2, d3):
        for i in range(min(counts[d1], counts[d2])):
            counts[d1] -= 1
            counts[d2] -= 1
            
            if d3:
                counts[d3] += 1
            
    steps = hex_path.split(",")
    counts = Counter(steps)
             
    equivalent("ne", "sw", None)
    equivalent("se", "nw", None)
    equivalent("s", "n", None)

    equivalent("ne", "nw", "n")
    equivalent("se", "sw", "s")
    equivalent("ne", "s", "se")
    equivalent("nw", "s", "sw")
    equivalent("sw", "n", "nw")
    equivalent("se", "n", "ne")
    return sum(counts.values())
        
assert shortest_path("ne,ne,ne") == 3
assert shortest_path("ne,ne,sw,sw") == 0
assert shortest_path("ne,ne,s,s") == 2
assert shortest_path("se,sw,se,sw,sw") == 3

shortest_path(Input(11).read().strip())

722

Whoops, can't reuse any of this for part 2.

In [22]:
class Location:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def move(self, direction):
        if direction == "n":
            self.y += 2
        if direction == "s":
            self.y -= 2
        if direction == "ne":
            self.x += 1
            self.y += 1
        if direction == "se":
            self.x += 1
            self.y -= 1
        if direction == "nw":
            self.x -= 1
            self.y += 1
        if direction == "sw":
            self.x -= 1
            self.y -= 1

def furthest_from_start(hex_path):
    loc = Location(0, 0)
    steps = hex_path.split(",")
    furthest = 0
    for step in steps:
        loc.move(step)
        furthest = max(furthest, (abs(loc.x) + abs(loc.y)) // 2)
    return furthest

assert furthest_from_start("ne,ne,sw,sw") == 2
furthest_from_start(Input(11).read().strip())

1551

# [Day 12](http://adventofcode.com/2017/day/12): Digital Plumber


In [23]:
test_input = """0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5"""
 
def piped(text, source):
    def add_deps(key):
        for e in pipes[key]:
            if not e in connected:
                connected.add(e)
                add_deps(e)
    pipes = {}
    connected = set()
    for line in text.split("\n"):
        key, _, *values = line.split(" ")
        values = set([v.strip(',') for v in values])
        pipes[key] = values
              
    add_deps(source)

    return len(connected)
 
assert piped(test_input, "0") == 6

piped(Input(12).read().strip(), "0")


145

In [24]:
def groups(text):
    def add_deps(key, connected=None):
        if not connected:
            connected = set()
        for e in pipes[key]:
            if not e in connected:
                connected.add(e)
                add_deps(e, connected)
        return connected
                
    pipes = {}
    groups = set()
    for line in text.split("\n"):
        key, _, *values = line.split(" ")
        values = set([v.strip(',') for v in values])
        pipes[key] = values
              
    groups = set()
    for k in pipes:
        deps = add_deps(k)
        groups.add(frozenset(deps))

    return len(groups)


assert groups(test_input) == 2
groups(Input(12).read().strip())


207

# [Day 13](http://adventofcode.com/2017/day/13): Packet Scanners

In [25]:
INPUT = """0: 4
1: 2
2: 3
4: 4
6: 8
8: 5
10: 6
12: 6
14: 10
16: 8
18: 6
20: 9
22: 8
24: 6
26: 8
28: 8
30: 12
32: 12
34: 12
36: 12
38: 10
40: 12
42: 12
44: 14
46: 8
48: 14
50: 12
52: 14
54: 14
58: 14
60: 12
62: 14
64: 14
66: 12
68: 12
72: 14
74: 18
76: 17
86: 14
88: 20
92: 14
94: 14
96: 18
98: 18"""

TEST_INPUT = """0: 3
1: 2
4: 4
6: 4"""

class Layer:
    def __init__(self, depth, range):
        self.depth = depth
        self.range = range
        self.scanner_location = 0
        self.up = lambda x: x + 1
        self.down = lambda x: x - 1
        self.scanner_move = self.up

    def bump_scanner(self):
        self.scanner_location = self.scanner_move(self.scanner_location)
        if self.scanner_location == 0:
            self.scanner_move = self.up
        if self.scanner_location == self.range - 1:
            self.scanner_move = self.down

class Firewall:
    def __init__(self, text):
        self.layers = {}
        self.packet = -1
        for line in text.splitlines():
            depth, range = [int(part.strip()) for part in line.split(":")]
            self.layers[depth] = Layer(depth, range)
            self.last = depth

    def reset_scanners(self):
        for layer in self.layers.values():
            layer.scanner_location = 0
            layer.scanner_move = layer.up
            layer.direction = "UP"
            
    def move_scanners(self):
        for layer in self.layers.values():
            layer.bump_scanner()

    def move_packet(self):
        self.packet += 1
        layer = self.layers.get(self.packet)
        loc = None
        if layer:
            loc = layer.scanner_location
        if layer and layer.scanner_location == 0:
            return layer.depth * layer.range
        return 0

    def cross(self):
        severity = 0
        for i in range(self.last + 1):
            severity += self.move_packet()
            self.move_scanners()
        return severity
            
f = Firewall(TEST_INPUT)
assert f.cross() == 24

f = Firewall(INPUT)
print("result", f.cross())


result 1844


Building the simulation above took way too long to compute part 2, so instead I'm now just calculating the position of the scanner at each timer, layer combo.

In [26]:
def find_delay(text):
    layers = {}
    for line in text.splitlines():
        depth, length = [int(part.strip()) for part in line.split(":")]
        layers[depth] = length
        
    for delay in count(0):
        if find_hit(delay, layers):
            return delay

def find_hit(delay, layers):
    highest = max(layers.keys())
    time = delay
    for tick in range(highest + 1):
        time = delay + tick
        layer = layers.get(tick)
        if layer and scanner_position(layer, time) == 0:
            return False
    return True
    
def scanner_position(length, time):
    position = time % (2 * length - 2)
    if position > length - 1:
        position = 2 * (length - 1) - position
    return position

find_delay(TEST_INPUT)
find_delay(INPUT)

3897604

# [Day 14](http://adventofcode.com/2017/day/14): Disk Defragmentation

In [27]:
TEST_INPUT = "flqrgnkx"
INPUT = "amgozmfv"
GRID_SIZE = 128

def disk_count(text):
    count = 0
    for i in range(GRID_SIZE):
        row_hash = knot_hash(text + "-" + str(i))
        count += bin(int(row_hash, 16))[2:].count("1")
    return count

assert disk_count(TEST_INPUT) == 8108
disk_count(INPUT)

8222

### Part Two
Now, all the defragmenter needs to know is the number of regions. A region is a group of used squares that are all adjacent, not including diagonals. Every used square is in exactly one region: lone used squares form their own isolated regions, while several adjacent squares all count as a single region.

In the example above, the following nine regions are visible, each marked with a distinct digit:
```
11.2.3..-->
.1.2.3.4   
....5.6.   
7.8.55.9   
.88.5...   
88..5..8   
.8...8..   
88.8.88.-->
|      |   
V      V   
```
Of particular interest is the region marked 8; while it does not appear contiguous in this small view, all of the squares marked 8 are connected when considering the whole 128x128 grid. In total, in this example, 1242 regions are present.

How many regions are present given your key string?

In [28]:
def disk_graph(text):
    grid = []
    for i in range(GRID_SIZE):
        row_hash = knot_hash(text + "-" + str(i))
        binary = bin(int(row_hash, 16))[2:]
        binary = (GRID_SIZE - len(binary)) * "0" + binary
        grid.append([int(i) for i in list(binary)])
    return grid

def bounded_neighbors(x, y, size):
    neighbors = []
    if x < size - 1:
        neighbors.append((x + 1, y))
    if y < size - 1:
        neighbors.append((x, y + 1))
    if x > 0:
        neighbors.append((x - 1, y))
    if y > 0:
        neighbors.append((x, y - 1))
    return neighbors

def dfs(x, y, grid):
    size = len(grid[0])
    visited = set((x, y))
    to_visit = bounded_neighbors(x, y, size)
    while to_visit:
        x, y = to_visit.pop()
        if grid[x][y] == 1 and (x, y) not in visited:
            visited.add((x, y))
            to_visit.extend(bounded_neighbors(x, y, size))
    
    return visited

def count_regions(grid, size):
    visited = set()
    regions = 0
    for i in range(size):
        for j in range(size):
            if (i, j) not in visited and grid[i][j] == 1:
                seen = dfs(i, j, grid)
                regions += 1
                visited = visited | seen
    return regions
                
TEST_GRID = [[1, 0, 1],
            [0, 0, 1],
            [1, 0, 1]]

assert count_regions(TEST_GRID, 3) == 3
assert count_regions(disk_graph(TEST_INPUT), GRID_SIZE) == 1242

count_regions(disk_graph(INPUT), GRID_SIZE)

1086

# [Day 15](http://adventofcode.com/2017/day/15): Dueling Generators
Here, you encounter a pair of dueling generators. The generators, called generator A and generator B, are trying to agree on a sequence of numbers. However, one of them is malfunctioning, and so the sequences don't always match.

As they do this, a judge waits for each of them to generate its next value, compares the lowest 16 bits of both values, and keeps track of the number of times those parts of the values match.

The generators both work on the same principle. To create its next value, a generator will take the previous value it produced, multiply it by a factor (generator A uses 16807; generator B uses 48271), and then keep the remainder of dividing that resulting product by 2147483647. That final remainder is the value it produces next.

To calculate each generator's first value, it instead uses a specific starting value as its "previous value" (as listed in your puzzle input).

For example, suppose that for starting values, generator A uses 65, while generator B uses 8921. Then, the first five pairs of generated values are:
```
--Gen. A--  --Gen. B--
   1092455   430625591
1181022009  1233683848
 245556042  1431495498
1744312007   137874439
1352636452   285222916
In binary, these pairs are (with generator A's value first in each pair):

00000000000100001010101101100111
00011001101010101101001100110111

01000110011001001111011100111001
01001001100010001000010110001000

00001110101000101110001101001010
01010101010100101110001101001010

01100111111110000001011011000111
00001000001101111100110000000111

01010000100111111001100000100100
00010001000000000010100000000100
```
Here, you can see that the lowest (here, rightmost) 16 bits of the third value match: 1110001101001010. Because of this one match, after processing these five pairs, the judge would have added only 1 to its total.

To get a significant sample, the judge would like to consider 40 million pairs. (In the example above, the judge would eventually find a total of 588 pairs that match in their lowest 16 bits.)

After 40 million pairs, what is the judge's final count?

In [29]:
A_START = 699
B_START = 124

A_FACTOR = 16807
B_FACTOR = 48271

DIVIDEND = 2147483647

def gen(previous, factor, dividend=DIVIDEND):
    return previous * factor % dividend


a = A_START
b = B_START
count = 0

for i in range(40*10**6):
    low_bits = lambda x: x & 2**16-1
    a = gen(a, A_FACTOR)
    b = gen(b, B_FACTOR)
    if low_bits(a) == low_bits(b):
        count += 1

count

600

In the interest of trying to align a little better, the generators get more picky about the numbers they actually give to the judge.

They still generate values in the same way, but now they only hand a value to the judge when it meets their criteria:

Generator A looks for values that are multiples of 4.
Generator B looks for values that are multiples of 8.
Each generator functions completely independently: they both go through values entirely on their own, only occasionally handing an acceptable value to the judge, and otherwise working through the same sequence of values as before until they find one.

The judge still waits for each generator to provide it with a value before comparing them (using the same comparison method as before). It keeps track of the order it receives values; the first values from each generator are compared, then the second values from each generator, then the third values, and so on.

Using the example starting values given above, the generators now produce the following first five values each:
```
--Gen. A--  --Gen. B--
1352636452  1233683848
1992081072   862516352
 530830436  1159784568
1980017072  1616057672
 740335192   412269392
 ```
These values have the following corresponding binary values:
```
01010000100111111001100000100100
01001001100010001000010110001000

01110110101111001011111010110000
00110011011010001111010010000000

00011111101000111101010001100100
01000101001000001110100001111000

01110110000001001010100110110000
01100000010100110001010101001000

00101100001000001001111001011000
00011000100100101011101101010000
```
Unfortunately, even though this change makes more bits similar on average, none of these values' lowest 16 bits match. Now, it's not until the 1056th pair that the judge finds the first match:
```
--Gen. A--  --Gen. B--
1023762912   896885216

00111101000001010110000111100000
00110101011101010110000111100000
```
This change makes the generators much slower, and the judge is getting impatient; it is now only willing to consider 5 million pairs. (Using the values from the example above, after five million pairs, the judge would eventually find a total of 309 pairs that match in their lowest 16 bits.)

After 5 million pairs, but using this new generator logic, what is the judge's final count?

In [30]:
A_START = 699
B_START = 124

A_FACTOR = 16807
B_FACTOR = 48271

DIVIDEND = 2147483647

def gen(previous, factor, multiple, dividend=DIVIDEND):
    candidate = previous
    while True:
        candidate = candidate * factor % dividend
        if candidate % multiple == 0:
            return candidate

a = A_START
b = B_START
count = 0

for i in range(5*10**6):
    low_bits = lambda x: x & 2**16-1
    a = gen(a, A_FACTOR, 4)
    b = gen(b, B_FACTOR, 8)
    if low_bits(a) == low_bits(b):
        count += 1

count

313

# [Day 16](https://adventofcode.com/2017/day/16): Permutation Promenade
You come upon a very unusual sight; a group of programs here appear to be dancing.

There are sixteen programs in total, named a through p. They start by standing in a line: a stands in position 0, b stands in position 1, and so on until p, which stands in position 15.

The programs' dance consists of a sequence of dance moves:

Spin, written sX, makes X programs move from the end to the front, but maintain their order otherwise. (For example, s3 on abcde produces cdeab).
Exchange, written xA/B, makes the programs at positions A and B swap places.
Partner, written pA/B, makes the programs named A and B swap places.
For example, with only five programs standing in a line (abcde), they could do the following dance:
```
s1, a spin of size 1: eabcd.
x3/4, swapping the last two programs: eabdc.
pe/b, swapping programs e and b: baedc.
```
After finishing their dance, the programs end up in order baedc.

You watch the dance for a while and record their dance moves (your puzzle input). In what order are the programs standing after their dance?

In [31]:
TEST_INPUT = "s1,x3/4,pe/b"

def dance_move(line, programs):
    def swap(a, b):
        temp = programs[a]
        programs[a] = programs[b]
        programs[b] = temp
    
    def spin(n):
        for i in range(n):
            programs.insert(0, programs.pop())
    
    move = line[0]
    
    if move == 's':
        spin(int(line[1:]))

    elif move == 'x':
        a, b = line[1:].split("/")
        swap(int(a), int(b))

    elif move == 'p':
        a, b = line[1:].split("/")
        swap(programs.index(a), programs.index(b))

def dance(text, end_program='p'):
    programs = [chr(i) for i in range(ord('a'),ord(end_program)+1)]
    for line in text.split(','):
        dance_move(line, programs)

    return "".join(programs)

assert dance(TEST_INPUT, 'e') == "baedc"
dance(Input(16).read().strip())

'iabmedjhclofgknp'

Now that you're starting to get a feel for the dance moves, you turn your attention to the dance as a whole.

Keeping the positions they ended up in from their previous dance, the programs perform it again and again: including the first dance, a total of one billion (1000000000) times.

In the example above, their second dance would begin with the order baedc, and use the same dance moves:
```
s1, a spin of size 1: cbaed.
x3/4, swapping the last two programs: cbade.
pe/b, swapping programs e and b: ceadb.
```

In what order are the programs standing after their billion dances?

In [32]:
def find_loop(text, end_program='p', n=10**9):
    programs = [chr(i) for i in range(ord('a'),ord(end_program)+1)]
    original = programs.copy()
    for i in range(n):
        for line in text.split(','):
            dance_move(line, programs)
            if programs == original:
                return i
find_loop(Input(16).read().strip())

35

In [33]:
1000000000 % 36

28

In [34]:
def many_dances(text, end_program='p', n=28):
    programs = [chr(i) for i in range(ord('a'),ord(end_program)+1)]
    for i in range(n):
        for line in text.split(','):
            dance_move(line, programs)
    return "".join(programs)

many_dances(Input(16).read().strip())

'oildcmfeajhbpngk'

# [Day 17](https://adventofcode.com/2017/day/17): Spinlock

Suddenly, whirling in the distance, you notice what looks like a massive, pixelated hurricane: a deadly spinlock. This spinlock isn't just consuming computing power, but memory, too; vast, digital mountains are being ripped from the ground and consumed by the vortex.

If you don't move quickly, fixing that printer will be the least of your problems.

This spinlock's algorithm is simple but efficient, quickly consuming everything in its path. It starts with a circular buffer containing only the value 0, which it marks as the current position. It then steps forward through the circular buffer some number of steps (your puzzle input) before inserting the first new value, 1, after the value it stopped on. The inserted value becomes the current position. Then, it steps forward from there the same number of steps, and wherever it stops, inserts after it the second new value, 2, and uses that as the new current position again.

It repeats this process of stepping forward, inserting a new value, and using the location of the inserted value as the new current position a total of 2017 times, inserting 2017 as its final operation, and ending with a total of 2018 values (including 0) in the circular buffer.

For example, if the spinlock were to step 3 times per insert, the circular buffer would begin to evolve like this (using parentheses to mark the current position after each iteration of the algorithm):

- (0), the initial state before any insertions.
- 0 (1): the spinlock steps forward three times (0, 0, 0), and then inserts the first value, 1, after it. 1 becomes the current position.
- 0 (2) 1: the spinlock steps forward three times (0, 1, 0), and then inserts the second value, 2, after it. 2 becomes the current position.
- 0  2 (3) 1: the spinlock steps forward three times (1, 0, 2), and then inserts the third value, 3, after it. 3 becomes the current position.
And so on:
```
0  2 (4) 3  1
0 (5) 2  4  3  1
0  5  2  4  3 (6) 1
0  5 (7) 2  4  3  6  1
0  5  7  2  4  3 (8) 6  1
0 (9) 5  7  2  4  3  8  6  1
```
Eventually, after 2017 insertions, the section of the circular buffer near the last insertion looks like this:

1512  1134  151 (2017) 638  1513  851
Perhaps, if you can identify the value that will ultimately be after the last value written (2017), you can short-circuit the spinlock. In this example, that would be 638.

What is the value after 2017 in your completed circular buffer?



In [35]:
INPUT = 345

def spinlock(steps=INPUT, insertions=2017):
    buffer = [0]
    index, value = 0, 1
    
    for i in range(insertions):
        index = (index + steps) % len(buffer) + 1
        buffer.insert(index, value)
        value += 1
        
    return buffer[(index + 1) % len(buffer)]

assert spinlock(3) == 638

spinlock(INPUT)

866

The spinlock does not short-circuit. Instead, it gets more angry. At least, you assume that's what happened; it's spinning significantly faster than it was a moment ago.

You have good news and bad news.

The good news is that you have improved calculations for how to stop the spinlock. They indicate that you actually need to identify the value after 0 in the current state of the circular buffer.

The bad news is that while you were determining this, the spinlock has just finished inserting its fifty millionth value (50000000).

What is the value after 0 the moment 50000000 is inserted?

In [37]:
def spinlock_part2(steps=INPUT, insertions=50000000):
    buffer = [0] * insertions
    index, value = 0, 1
    
    for i in range(insertions):
        index = (index + steps) % value + 1
        buffer[index] = value
        value += 1
        
    return buffer[(buffer.index(0) + 1) % len(buffer)]

%time spinlock_part2()

CPU times: user 17 s, sys: 712 ms, total: 17.7 s
Wall time: 17.7 s


11995607

# [Day 18](https://adventofcode.com/2017/day/18): Duet

You discover a tablet containing some strange assembly code labeled simply "Duet". Rather than bother the sound card with it, you decide to run the code yourself. Unfortunately, you don't see any documentation, so you're left to figure out what the instructions mean on your own.

It seems like the assembly is meant to operate on a set of registers that are each named with a single letter and that can each hold a single integer. You suppose each register should start with a value of 0.

There aren't that many instructions, so it shouldn't be hard to figure out what they do. Here's what you determine:
```
snd X plays a sound with a frequency equal to the value of X.
set X Y sets register X to the value of Y.
add X Y increases register X by the value of Y.
mul X Y sets register X to the result of multiplying the value contained in register X by the value of Y.
mod X Y sets register X to the remainder of dividing the value contained in register X by the value of Y (that is, it sets X to the result of X modulo Y).
rcv X recovers the frequency of the last sound played, but only when the value of X is not zero. (If it is zero, the command does nothing.)
jgz X Y jumps with an offset of the value of Y, but only if the value of X is greater than zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)
```
Many of the instructions can take either a register (a single letter) or a number. The value of a register is the integer it contains; the value of a number is that number.

After each jump instruction, the program continues with the instruction to which the jump jumped. After any other instruction, the program continues with the next instruction. Continuing (or jumping) off either end of the program terminates it.

For example:
```
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
```
The first four instructions set a to 1, add 2 to it, square it, and then set it to itself modulo 5, resulting in a value of 4.
Then, a sound with frequency 4 (the value of a) is played.
After that, a is set to 0, causing the subsequent rcv and jgz instructions to both be skipped (rcv because a is 0, and jgz because a is not greater than 0).
Finally, a is set to 1, causing the next jgz instruction to activate, jumping back two instructions to another jump, which jumps again to the rcv, which ultimately triggers the recover operation.
At the time the recover operation is executed, the frequency of the last sound played is 4.

What is the value of the recovered frequency (the value of the most recently played sound) the first time a rcv instruction is executed with a non-zero value?



In [89]:
INPUT = """set i 31
set a 1
mul p 17
jgz p p
mul a 2
add i -1
jgz i -2
add a -1
set i 127
set p 464
mul p 8505
mod p a
mul p 129749
add p 12345
mod p a
set b p
mod b 10000
snd b
add i -1
jgz i -9
jgz a 3
rcv b
jgz b -1
set f 0
set i 126
rcv a
rcv b
set p a
mul p -1
add p b
jgz p 4
snd a
set a b
jgz 1 3
snd b
set f 1
add i -1
jgz i -11
snd a
jgz f -16
jgz a -19"""

In [122]:
TEST = """set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2"""

def execute(command, regs):
    # Return (position delta, sound). delta == None indicates termination.
    def parse(arg):
        if arg.isalpha():
            return arg
        return int(arg)
    
    instruction, args = command.split(" ")[0], command.split(" ")[1:]
    x = parse(args[0])
    if len(args) > 1:
        y = parse(args[1])
        if not type(y) == int:
            y = regs[y]

    if instruction == "snd": return (1, regs[x])
    elif instruction == "set": regs[x] = y
    elif instruction == "add": regs[x] = regs[x] + y
    elif instruction == "mul": regs[x] = regs[x] * y
    elif instruction == "mod": regs[x] = regs[x] % y
    elif instruction == "rcv" and regs[x]: return (None, None) 
    elif instruction == "jgz" and regs[x]: return (y, None)
    
    return (1, None)

def run(text):    
    regs = defaultdict(int)
    last, position, delta = 0, 0, 1
    commands = text.split("\n")
    
    while True:
        command = commands[position]
        delta, sound = execute(command, regs)
        if sound:
            last = sound
        if not delta:
            return last
        position += delta
             
assert run(TEST) == 4

run(INPUT)

1187

### Part Two
As you congratulate yourself for a job well done, you notice that the documentation has been on the back of the tablet this entire time. While you actually got most of the instructions correct, there are a few key differences. This assembly code isn't about sound at all - it's meant to be run twice at the same time.

Each running copy of the program has its own set of registers and follows the code independently - in fact, the programs don't even necessarily run at the same speed. To coordinate, they use the send (snd) and receive (rcv) instructions:

snd X sends the value of X to the other program. These values wait in a queue until that program is ready to receive them. Each program has its own message queue, so a program can never receive a message it sent.
rcv X receives the next value and stores it in register X. If no values are in the queue, the program waits for a value to be sent to it. Programs do not continue to the next instruction until they have received a value. Values are received in the order they are sent.
Each program also has its own program ID (one 0 and the other 1); the register p should begin with this value.

For example:
```
snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
```
Both programs begin by sending three values to the other. Program 0 sends 1, 2, 0; program 1 sends 1, 2, 1. Then, each program receives a value (both 1) and stores it in a, receives another value (both 2) and stores it in b, and then each receives the program ID of the other program (program 0 receives 1; program 1 receives 0) and stores it in c. Each program now sees a different value in its own copy of register c.

Finally, both programs try to rcv a fourth time, but no data is waiting for either of them, and they reach a deadlock. When this happens, both programs terminate.

It should be noted that it would be equally valid for the programs to run at different speeds; for example, program 0 might have sent all three values and then stopped at the first rcv before program 1 executed even its first instruction.

Once both of your programs have terminated (regardless of what caused them to do so), how many times did program 1 send a value?

In [170]:
def execute(command, regs, sndq, rcvq):
    """Returns (position delta, send count)"""
    
    def parse(arg):
        """Converts numeric args to ints"""
        if arg.isalpha():
            return arg
        return int(arg)
    
    def value(arg):
        """If an arg is not numeric, return the value contained in the register[arg]"""
        if not type(arg) == int:
             return regs[arg]
        return arg
    
    instruction, args = command.split(" ")[0], command.split(" ")[1:]
    x = parse(args[0])
    if len(args) > 1:
        y = value(parse(args[1]))
    
    if instruction == "snd":
        x = value(x)
        sndq.insert(0, x)
        return (1, 1)
    elif instruction == "rcv" and regs[x]:
        if rcvq: 
            regs[x] = rcvq.pop()
        else: return (0, 0)
        
    elif instruction == "set": regs[x] = y
    elif instruction == "add": regs[x] = regs[x] + y
    elif instruction == "mul": regs[x] = regs[x] * y
    elif instruction == "mod": regs[x] = regs[x] % y
    elif instruction == "jgz":
        x = value(x)
        if x > 0:
            return (y, 0)
    
    return (1, 0)

def run(text):    
    regs_a, regs_b = defaultdict(int), defaultdict(lambda: 1)
    position_a, position_b = 0, 0
    a_sndq, b_sndq = [], []
    b_send = 0
    commands = text.split("\n")
    
    a_term, b_term = False, False
    while not a_term and not b_term:
        if not a_term:
            command_a = commands[position_a]
            delta_a, _ = execute(command_a, regs_a, a_sndq, b_sndq)
            position_a += delta_a
        
        if not b_term:
            command_b = commands[position_b]

            delta_b, send = execute(command_b, regs_b, b_sndq, a_sndq)
            b_send += send
            position_b += delta_b

        if delta_a == 0 and delta_b == 0:
            return b_send

        if position_a >= len(commands) or position_a < 0:
            a_term = True
        
        if position_b >= len(commands) or position_b < 0:
            b_term = True

    return b_send

run(INPUT)

5969

# [Day 19](https://adventofcode.com/2017/day/19): A Series of Tubes
Somehow, a network packet got lost and ended up here. It's trying to follow a routing diagram (your puzzle input), but it's confused about where to go.

Its starting point is just off the top of the diagram. Lines (drawn with |, -, and +) show the path it needs to take, starting by going down onto the only line connected to the top of the diagram. It needs to follow this path until it reaches the end (located somewhere within the diagram) and stop there.

Sometimes, the lines cross over each other; in these cases, it needs to continue going the same direction, and only turn left or right when there's no other option. In addition, someone has left letters on the line; these also don't change its direction, but it can use them to keep track of where it's been. For example:

```
     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ 
```
Given this diagram, the packet needs to take the following path:

Starting at the only line touching the top of the diagram, it must go down, pass through A, and continue onward to the first +.
Travel right, up, and right, passing through B in the process.
Continue down (collecting C), right, and up (collecting D).
Finally, go all the way left through E and stopping at F.
Following the path to the end, the letters it sees on its path are ABCDEF.

The little packet looks up at you, hoping you can help it find the way. What letters will it see (in the order it would see them) if it follows the path? (The routing diagram is very wide; make sure you view it without line wrapping.)

In [237]:
TEST = """     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ """

def get_neighbors(x, y, grid):
    neighbors = []
    max_x = len(grid[0])
    max_y = len(grid)
    
    if x > 0:
        neighbors.append((x - 1, y))
    if x < max_x - 1:
        neighbors.append((x + 1, y))
    if y > 0:
        neighbors.append((x, y -1))
    if y < max_y - 1:
        neighbors.append((x, y + 1))
    return neighbors

def traverse(text):
    up = lambda x, y: (x, y + 1)
    down = lambda x, y: (x, y - 1)
    left = lambda x, y: (x - 1, y)
    right = lambda x, y: (x + 1, y)
    
    def new_direction(x, y, old_x, old_y):
        if x == old_x:
            if y - old_y > 0:
                return up
            return down
        if x - old_x > 0:
                return right
        return left
    
    grid = [[c for c in line] for line in text.split("\n")]
    x, y = grid[0].index('|'), 0
    move = up
    path = []
    previous = None
    steps = 0
    
    while True:
        symbol = grid[y][x]
        if symbol == " ":
            return "".join(path), steps
        
        steps += 1
    
        if symbol in ('|', '-'):
            previous = (x, y)
            x, y = move(x, y)
            
        elif symbol == '+':
            candidates = get_neighbors(x, y, grid)
            candidates.remove(previous)
            previous = (x, y)

            for new_x, new_y in candidates:
                if grid[new_y][new_x] != ' ':
                    move = new_direction(new_x, new_y, x, y)
                    x, y = move(x, y)
                    
            if (x, y) == previous:
                return "".join(path), steps
    
        else:
            path.append(symbol)
            previous = (x, y)
            x, y = move(x, y)
    
assert traverse(TEST) == ('ABCDEF', 38)

traverse(Input(19).read())

('KGPTMEJVS', 16328)

# [Day 20](https://adventofcode.com/2017/day/20): A Series of Tubes
Suddenly, the GPU contacts you, asking for help. Someone has asked it to simulate too many particles, and it won't be able to finish them all in time to render the next frame at this rate.

It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the X, Y, and Z coordinates for the particle's position (p), velocity (v), and acceleration (a), each in the format <X,Y,Z>.

Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:
```
Increase the X velocity by the X acceleration.
Increase the Y velocity by the Y acceleration.
Increase the Z velocity by the Z acceleration.
Increase the X position by the X velocity.
Increase the Y position by the Y velocity.
Increase the Z position by the Z velocity.
```
Because of seemingly tenuous rationale involving z-buffering, the GPU would like to know which particle will stay closest to position <0,0,0> in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's X, Y, and Z position.

For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). Drawing the current states of particles 0 and 1 (in that order) with an adjacent a number line and diagram of current X positions (marked in parenthesis), the following would take place:
```
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                         (0)(1)

p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                      (1)   (0)

p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>          (1)               (0)

p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                         (0)   
```
At this point, particle 1 will never be closer to <0,0,0> than particle 0, and so, in the long run, particle 0 will stay closest.

Which particle will stay closest to position <0,0,0> in the long term?

In [334]:
TEST = """p=<3,0,0>, v=<2,0,0>, a=<-1,0,0>
p=<4,0,0>, v=<0,0,0>, a=<-2,0,0>"""

class Particle:
    def __init__(self, position, velocity, acceleration):
        self.position = position
        self.velocity = velocity
        self.acceleration = acceleration
        
    def tick(self):
        for i, component in enumerate(self.acceleration): self.velocity[i] += component
        for i, component in enumerate(self.velocity): self.position[i] += component

        self.distance = sum((abs(component) for component in self.position))
          
def parse(line):
    return Particle(*[[int(e) for e in v[v.find("<") + 1 : v.find(">")].split(",")] for v in line.split(" ")])
        
def simulate(text, ticks=1000):
    particles = [parse(line) for line in text.split("\n")]
    for i in range(ticks):
        for particle in particles:
            particle.tick()
            
    return particles.index(min(particles, key=lambda p:p.distance))
        
assert simulate(TEST) == 0

simulate(Input(20).read().strip())

300

### Part Two 
To simplify the problem further, the GPU would like to remove any particles that collide. Particles collide if their positions ever exactly match. Because particles are updated simultaneously, more than two particles can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.

For example:
```
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>    (0)   (1)   (2)            (3)
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=<-3,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-2,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-1,0,0>, v=< 1,0,0>, a=< 0,0,0>             (0)(1)(2)      (3)   
p=< 2,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=< 0,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=< 0,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=< 0,0,0>, v=< 1,0,0>, a=< 0,0,0>                       X (3)      
p=< 1,0,0>, v=<-1,0,0>, a=< 0,0,0>

------destroyed by collision------    
------destroyed by collision------    -6 -5 -4 -3 -2 -1  0  1  2  3
------destroyed by collision------                      (3)         
p=< 0,0,0>, v=<-1,0,0>, a=< 0,0,0>
```
In this example, particles 0, 1, and 2 are simultaneously destroyed at the time and place marked X. On the next tick, particle 3 passes through unharmed.

How many particles are left after all collisions are resolved?

In [344]:
def remove_collisions(particles):
    seen_positions = defaultdict(list)
    for particle in particles:
        seen_positions[tuple(particle.position)].append(particle)
    
    for v in seen_positions.values():
        if len(v) > 1:
            for e in v:
                particles.remove(e)
    
def simulate2(text, ticks=1000):
    particles = [parse(line) for line in text.split("\n")]
    for i in range(ticks):
        for particle in particles:
            particle.tick()
        remove_collisions(particles)
            
    return len(particles)

TEST="""p=<-6,0,0>, v=<3,0,0>, a=<0,0,0>
p=<-4,0,0>, v=<2,0,0>, a=<0,0,0>
p=<-2,0,0>, v=<1,0,0>, a=<0,0,0>
p=<3,0,0>, v=<-1,0,0>, a=<0,0,0>"""
assert simulate2(TEST) == 1

simulate2(Input(20).read().strip())

502